<a href="https://colab.research.google.com/github/sivasurasani/Material-Science-Research/blob/main/Hollomon_constants_json_appender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
# Mount Google Drive
drive.mount('/content/drive')

# Directory containing all the data folders
data_directory = "/content/drive/MyDrive/Colab Notebooks/Material_Science/H_Data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
def calculate_hollomon_constants(strain, stress):
    # Filter out non-positive values
    valid_indices = (strain > 0) & (stress > 0)
    strain = strain[valid_indices]
    stress = stress[valid_indices]

    # Transforming the data to natural logarithms
    log_strain = np.log(strain)
    log_stress = np.log(stress)

    # Performing linear regression
    X = sm.add_constant(log_strain)  # Adds a constant term to the predictor
    model = sm.OLS(log_stress, X).fit()
    intercept, slope = model.params

    # Calculating k
    k = np.exp(intercept)

    return slope, k

def plot_stress_strain(strain, stress, n, k):
    # Generate fitted stress values using the Hollomon equation
    fitted_stress = k * (strain**n)
    # Plot original data
    plt.scatter(strain, stress, color='blue', label='Original data')
    # Plot fitted curve
    plt.plot(strain, fitted_stress, color='red', label=f'Fitted curve: $\sigma = {k:.2f} \cdot \epsilon^{n:.2f}$')
    plt.xlabel('Strain')
    plt.ylabel('Stress')
    plt.legend()
    plt.title('Stress vs. Strain')
    plt.show()

# data = pd.read_excel('Eng Values.xlsx')
# if 't.stress' not in data.columns and 't.strain' not in data.columns:
#     data['t.stress'] = data['eng.stress'] * (1 + data['eng.strain'])
#     data['t.strain'] = np.log(1 + data['eng.strain'])
# stress = data['t.stress']
# strain = data['t.strain']
# # stress = fitted_true_stress
# # strain= new_true_strain

# try:
#     n, k = calculate_hollomon_constants(strain, stress)
#     print(f'n = {n}')
#     print(f'k = {k}')
#     plot_stress_strain(strain, stress, n, k)
# except ValueError as e:
#     print(e)

In [17]:
# Loop through each folder in the data directory
json_file_path = 'data.json'
import json
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
papers_dict = {paper['title']: paper for paper in json_data}
for folder_name in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder_name)

    if not os.path.isdir(folder_path):
        print(f"Skipping file: {folder_name}, it is not a folder.")
        continue

    print(f"\nPROCESSING FOLDER: {folder_name}")

    # Find the Excel file in the folder
    excel_file = next((os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".xlsx")), None)

    if excel_file:
        print(f"Found Excel file: {excel_file}")

        # Read Excel data
        data = pd.read_excel(excel_file, usecols=[0, 1])

        # Check and calculate the t.stress and t.strain columns
        if 't.stress' not in data.columns or 't.strain' not in data.columns:
            data['t.stress'] = data['eng.stress'] * (1 + data['eng.strain'])
            data['t.strain'] = np.log(1 + data['eng.strain'])

        stress = data['t.stress']
        strain = data['t.strain']

        # Calculate Hollomon constants
        n, k = calculate_hollomon_constants(strain, stress)
        print(f'n = {n}')
        print(f'k = {k}')

        # Update the corresponding JSON entry using the dictionary
        if folder_name in papers_dict:
            papers_dict[folder_name]['n'] = n
            papers_dict[folder_name]['k'] = k
        else:
            print(f"Error: No matching title found for folder {folder_name} in JSON data.")
    else:
        print(f"Error: Excel file is required in folder {folder_name}.")

# Convert the dictionary back to a list
json_data_updated = list(papers_dict.values())

# Save the updated JSON data back to the file
with open(json_file_path, 'w') as file:
    json.dump(json_data_updated, file, indent=4)

print("Updated JSON data saved to data.json")


PROCESSING FOLDER: paper-1
Found Excel file: /content/drive/MyDrive/Colab Notebooks/Material_Science/H_Data/paper-1/Eng Values.xlsx
n = 0.37546100095031515
k = 2062.1744509055993

PROCESSING FOLDER: paper-5
Found Excel file: /content/drive/MyDrive/Colab Notebooks/Material_Science/H_Data/paper-5/Eng Values.xlsx
n = 0.3364048615463518
k = 2136.9093521940476
Updated JSON data saved to data.json
